In [1]:
# https://stackoverflow.com/questions/37159070/multiple-linear-regression-model-by-using-tensorflow
# https://donaldpinckney.com/books/pytorch/book/ch2-linreg/2018-03-21-multi-variable.html
# https://www.youtube.com/watch?v=Q4GNLhRtZNc
# https://atmamani.github.io/projects/ml/coursera-gd-multivariate-linear-regression/
# https://online.stat.psu.edu/stat462/sites/onlinecourses.science.psu.edu.stat462/files/05mlr/eq_matrix_notation/index.gif

# Multivariable Logistic Regression for matricies.
# target = flux1 + flux2 +... flux500 + b
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 
import numpy as np
import pandas as pd
from numpy import exp
from sklearn import preprocessing
import pickle

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
def labels(filename):
    with open(filename, 'rb') as f:
        labels = np.load(f).transpose()
    return labels

In [3]:
#Logistic Layer using a sigmoid function
def logistic_layer(y):
    y = np.array(y)
    y = 1 / (1 + exp(-y)) # sigmoid function
    y = y.ravel()
    return y

In [4]:
#Calculate an accuracy metric
def accuracy(predicted_y, true_y):
    true_y = np.array(true_y).ravel()
    counter = 0
    for i in range(len(true_y)):
        p_y = predicted_y[i]
        t_y = true_y[i]
        if (p_y>.5 and t_y == 1) or (p_y < .5 and t_y == 0):
            counter+=1
    counter = (counter/ len(true_y)) * 100
    return counter

In [5]:
training_data_x = pickle.load(open("TS-Train.pkl", "rb"))
print(training_data_x.columns[training_data_x.isna().any()].tolist())
training_data_x = training_data_x.dropna(axis=1)
training_data_x = preprocessing.normalize(training_data_x,norm='max', axis=0)
training_data_x = training_data_x.transpose()
print(training_data_x.shape)
training_data_y = labels("../Labels-Train.npy")
print(training_data_y)

#set hyperparameters & variables
learning_rate = 0.03
epochs = 500
display_step = 5
n_samples = training_data_x.shape[1]
col_num = training_data_x.shape[0]

X = tf.placeholder(tf.float32, [col_num, n_samples])
Y = tf.placeholder(tf.float32, [1, n_samples]) #resulting dimenstion of W*X matmul

[]
(769, 11021)
[[0 0 0 ... 1 0 0]]


In [6]:
# We want the weight vector to correspond one to one with every column
W = tf.Variable(tf.zeros([1,col_num], dtype=np.float32), name="weight")
b = tf.Variable(tf.zeros([1, ], dtype=np.float32), name="bias")

#matrix multiplication requires outer dimension of W to be equal to be equal to the inner dimension of X: 
# (1,col_num) & (col_num, num_samples) - this is why we transpose X
pred = tf.matmul(W, X) + b # y′(x,A,b)=Ax+b linear matrix equation

error = tf.reduce_sum((pred-Y)**2) / (n_samples * 2) #MSE

In [7]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(error)

session = tf.Session()
session.run(tf.global_variables_initializer())

loss_arr = []
acc_arr = []

for t in range(epochs):
    
    _, current_loss, current_W, current_b = session.run([optimizer, error, W, b], feed_dict={
        X: training_data_x,
        Y: training_data_y
    })
    
    loss_arr.append(current_loss)
    acc_arr.append(accuracy(logistic_layer(np.dot(current_W,training_data_x) + current_b), training_data_y))
    
print("Optimization Finished!")

training_error = session.run(error, feed_dict={X: training_data_x, Y: training_data_y})
print("Training error=", training_error, "Weights=", session.run(W), "Bias=", session.run(b), '\n')

2021-12-13 02:42:04.435644: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Optimization Finished!
Training error= 0.044331145 Weights= [[ 5.36810420e-03  1.96958054e-02  2.27790363e-02 -7.78556755e-03
  -1.08380034e-03  5.37627377e-04 -2.18205643e-03 -3.80541175e-03
   3.87997925e-03  0.00000000e+00 -1.08380034e-03 -8.20909441e-03
  -1.16759213e-03  8.42946029e-05  4.36961331e-04 -1.71008296e-02
  -6.47426769e-03 -9.60198231e-04 -2.18205643e-03 -2.54432820e-02
  -7.95905292e-02  1.78979468e-02 -2.71927789e-02  8.05565994e-03
   8.25574528e-03 -1.84296258e-03 -1.46355527e-03 -4.84247552e-03
  -7.24272977e-04 -7.63475290e-03 -8.98448471e-03 -3.14770523e-03
  -1.70817412e-03  8.20909441e-03 -1.34884855e-02  1.81160765e-04
  -1.70708503e-04 -4.40863150e-05  3.33903969e-04  3.79961275e-04
   4.40937496e-04  9.16013774e-03 -9.92591609e-04  0.00000000e+00
  -4.53061201e-02  5.82277924e-02 -4.24376968e-03 -8.20729695e-03
  -8.20909441e-03 -8.20909441e-03 -8.20909441e-03 -8.20909441e-03
  -8.20909441e-03 -8.20909441e-03 -8.20909441e-03 -8.20909441e-03
  -8.20909441e-0

In [8]:
test_x = pickle.load(open("TS-Test.pkl", "rb"))
print(test_x.columns[test_x.isna().any()].tolist())
test_x = test_x.dropna(axis=1)
test_x = preprocessing.normalize(test_x, norm='max', axis=0)
test_x = test_x.transpose()

print(test_x.shape)
test_y = labels("../Labels-Test.npy")

predicted_y = np.dot(session.run(W), test_x) + session.run(b)
predicted_y = logistic_layer(predicted_y)

print("Accuracy percentage: ", accuracy(predicted_y, test_y), "%")

[]
(769, 6)
Accuracy percentage:  66.66666666666666 %


In [9]:
with open('tf-models/multi-lr.npy', 'wb') as f:
    np.save(f, session.run(W))
    np.save(f, session.run(b))
    np.save(f, loss_arr)
    np.save(f, acc_arr)
